In [ ]:
#encoding=utf-8
import jieba
import collections
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from math import log10
import matplotlib.font_manager as fm

# jieba.set_dictionary('./dict.txt')
# warnings.filterwarnings('ignore')

In [2]:
def preprocessing(articles):
    dataset = list()
    for article in articles[:1000]: # 先少一點
        sentences = article[:-1].split('\t') # sentences: list
        for i in range(len(sentences)): # iterate 時不會直接改到原本容器, 所以要用 index traverse
            sentences[i] = re.sub(r'\W+', "", sentences[i])
        parsed_sentences = list()
        for sentence in sentences:
            words = list(jieba.cut(sentence, cut_all=False)) # 一句一句切
            parsed_sentences.append(words) # 切過的句子組合成文章
        # print(parsed_sentences)
        dataset.append(parsed_sentences) # 文章組合成檔案集 # parsed_sentences = article
    # print(dataset)
    return list(dataset)

In [3]:
def tf(article): # t: 詞
    word_statistics_in_article = collections.Counter()
    for sentence in article:
        word_statistics_in_article += collections.Counter(sentence)
    # print(word_statistics_in_article)
    ### 計算 tf
    total = sum(word_statistics_in_article.values(), 0.0)
    word_tf_in_article = {key: round((val/total), 2) for key, val in sorted(word_statistics_in_article.items(), key=lambda x: x[1], reverse=True)}
    # print(pd.DataFrame(list(word_tf_in_article.items()), columns=['word', 'tf'])) # DataFrame 好看但我不太會用
    return word_tf_in_article

In [4]:
def idf(tf_per_article_table, q): # terms: list of all words
    word_appear_count = collections.Counter()
    for i in range(len(tf_per_article_table)):
        word_appear_count += collections.Counter({key: 1 for key in tf_per_article_table[i].keys()})
    
    idf_table = {word: round(log10(q / count), 3) for word, count in word_appear_count.items()}
    return idf_table

In [40]:
from matplotlib.font_manager import findfont, FontProperties
findfont(FontProperties(family=FontProperties().get_family()))

'/Users/jonahtsao/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'

In [6]:
def word_tf_in_dataset(dataset):
    all_words_count = collections.Counter()
    for article in dataset:
        for sentence in article:
            all_words_count += collections.Counter(sentence)
    total = sum(all_words_count.values(), 0.0)
    all_words_tf = {key: round((val/total), 3) for key, val in sorted(all_words_count.items(), key=lambda x: x[1], reverse=True)}
    return all_words_tf

In [8]:
def tf_idf(tf_t_d, idf_t_D):
    ### To Do
    # print(idf_t_D)
    table = dict()
    for t, tf in tf_t_d.items():
        # print(t, tf, idf_t_D[t])
        table[t] = round((tf * idf_t_D[t]), 3)
    # print(table)
    return {key: val for key, val in sorted(table.items(), key=lambda x: x[1], reverse=True)}

In [70]:
def rank_100(tf_idf_table):
    skip_flag = {key: False for key in tf_idf_table.keys()}
    rank_100 = {'a': 0.0}
    min_in_rank = ('a', 0.0)
    idx = 0
    while True:
        stop_flag = True
        for i in range(len(tf_idf_table)):
            if idx >= len(tf_idf_table[i].items()): # 第 i 篇文章已經掃完了, 直接往下一篇
                continue
            if skip_flag[i] != True: # 第 i 篇文章還沒掃完, 且不 skip
                word, weight = tuple(tf_idf_table[i].items())[idx] # list(tf_idf_table[i].items())[idx] 第 i 個文章中的 第 idx 大的
                if weight > min_in_rank[1]:  # if 文章 i 第一高權重的值 > rank 100 中最小值
                    rank_100[ word ] = weight # 加進新血
                    if len(rank_100) > 100: # 若列表超過 100, 該踢人 (ps: 還沒滿的話不排也沒差)
                        rank_100.pop(min_in_rank[0]) # 殺掉最小
                        new_rank_list = sorted(rank_100.items(), key=lambda x: x[1], reverse=True) # 重新排序後的列表
                        min_in_rank = new_rank_list[-1] # 更新最小
                        rank_100 = {key: val for key, val in new_rank_list} # 用重排後的 list 更新 rank_100
                    
                    stop_flag = False
                else:
                    skip_flag[i] = True
                
        if stop_flag == True: break # 都沒更新 === skip_flag 全部變 True 了, 跳出迴圈
        idx += 1
    # print(rank_100, len(rank_100))
    return list(rank_100.items())

In [65]:
def draw(rank_100, filename, row_labels, col_labels):
    # print(rank_100)
    x = [item[0] for item in rank_100]
    y = [item[1] for item in rank_100]
    fontPath = r'/Users/jonahtsao/Desktop/七年/tools/NotoSansCJK-TC/NotoSansCJKtc-Light.otf'
    font = fm.FontProperties(fname=fontPath, size=10)
    
    fig = plt.figure(figsize=(20,20))
    plt.bar(x, y)
    plt.xticks(range(len(x)), x, fontproperties=font)
    plt.title(filename)
    plt.xlabel(row_labels)
    plt.ylabel(col_labels)
    
    plt.savefig("./" + filename + ".png")
    return None

In [8]:
def main():
    file = open("./hw1-dataset.txt", mode='r')
    articles = file.readlines()
    dataset = preprocessing(articles)
    # print(dataset)
    ### 統計高頻
    tf_whole_dataset_table = word_tf_in_dataset(dataset) # 高頻詞(已排序)
    # print(list(tf_whole_dataset_table.items())[:100])
    tf_100 = list(tf_whole_dataset_table.items())[:100]
    
    ### 建 tf table
    tf_per_article_table = dict()
    for article_no in range(len(dataset)):
        tf_per_article_table[article_no] = tf(dataset[article_no])
        print(tf_per_article_table[article_no]) # 取值: tf_per_article_table[article_no][t] = tf(t, d)
    
    ### 建 idf table
    idf_table = idf(tf_per_article_table, len(dataset))
    # print(idf_table)
    
    ### 統計 tf-idf 高
    tf_idf_table = dict()
    for article_no in range(len(dataset)):
        tf_idf_table[article_no] = tf_idf(tf_per_article_table[article_no], idf_table)
    # print(tf_idf_table)
    tf_idf_100 = rank_100(tf_idf_table)
    
    ### 畫圖
    draw(tf_100, "tf_100", "term", "frequency")
    draw(tf_idf_100, "tf_idf_100", "term", "weight")
    
    file.close()
main()

{'這群': 0.08, '人': 0.08, '為': 0.04, '什麼': 0.04, '聖結': 0.04, '石會': 0.04, '被': 0.04, '酸': 0.04, '而': 0.04, '不會': 0.04, '質感': 0.04, '劇本': 0.04, '成員': 0.04, '都': 0.04, '差': 0.04, '很多': 0.04, '好': 0.04, '嗎': 0.04, '不要': 0.04, '拿': 0.04, '腎': 0.04, '結石來': 0.04, '污辱': 0.04}
{'為': 0.06, '什麼': 0.06, '慶祝': 0.06, '228': 0.06, '會': 0.06, '被': 0.06, '罵': 0.06, '可是': 0.06, '慶': 0.06, '端午': 0.06, '不會': 0.06, '因為': 0.06, '屈原': 0.06, '不是': 0.06, '台灣': 0.06, '人': 0.06, '是': 0.06, '楚國人': 0.06}
{'有': 0.15, '阿瑞斯': 0.15, '沒': 0.08, '戰神': 0.08, '的': 0.08, '八卦': 0.08, '爵士': 0.08, '就是': 0.08, '男主角': 0.08, '最後死': 0.08, '了': 0.08}
{'脫節': 0.16, '最': 0.11, '的': 0.11, '系': 0.11, '不': 0.11, '理論': 0.05, '與': 0.05, '實務': 0.05, '哪個': 0.05, '你問': 0.05, '簡單': 0.05, '多': 0.05, '了': 0.05}
{'看': 0.13, '棒球': 0.13, '為': 0.07, '什麼': 0.07, 'PTT': 0.07, '這麼': 0.07, '多': 0.07, '人': 0.07, '肥宅': 0.07, '才': 0.07, '系壘': 0.07, '一堆': 0.07, '胖子': 0.07}
{'為': 0.06, '什麼': 0.06, '達摩祖': 0.06, '師傳': 0.06, '那麼': 0.06, '好看': 0.06, '達摩': 0.06, '